In [1]:
from rightmove_scraper import RightmoveScraper, get_regions
import pandas as pd

In [2]:
regions = get_regions()

rms = RightmoveScraper(True)
buy_df = rms.scrape_regions(regions, save=True, verbose=True)

rms = RightmoveScraper(False)
rent_df = rms.scrape_regions(regions, save=True, verbose=True)


Balham:   0%|          | 0/65 [00:00<?, ?it/s]

In [113]:
df = pd.concat([buy_df, rent_df]).drop('LINK', axis=1)
df.set_index('ID', inplace=True)
group = df.groupby('MODE').mean()

/var/folders/qj/6bfpb5dj423_qz_zy21j4rkm0000gn/T/ipykernel_27065/503491351.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  group = df.groupby('MODE').mean()


In [82]:
group.loc['RENT'] / group.loc['BUY'] 

BEDROOMS     0.960150
BATHROOMS    0.932697
PRICE        0.003264
dtype: float64

In [116]:
avg = df.groupby(['BEDROOMS', 'MODE', 'PROPERTY TYPE']).mean()
median = df.groupby(['BEDROOMS', 'MODE']).median()

count = df.groupby(['BEDROOMS', 'MODE']).count()['PROPERTY TYPE']
grouped = pd.concat([avg, median ,count], keys=['avg', 'median', 'count'], axis=1)

/var/folders/qj/6bfpb5dj423_qz_zy21j4rkm0000gn/T/ipykernel_27065/1621188667.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg = df.groupby(['BEDROOMS', 'MODE', 'PROPERTY TYPE']).mean()
/var/folders/qj/6bfpb5dj423_qz_zy21j4rkm0000gn/T/ipykernel_27065/1621188667.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  median = df.groupby(['BEDROOMS', 'MODE']).median()


ValueError: Length of names must match number of levels in MultiIndex.

In [119]:
avg[avg.index.get_level_values('BEDROOMS')==1].round(2)

BATHROOMS       PRICE
BEDROOMS MODE PROPERTY TYPE                             
1.0      BUY  Apartment                 1.00   931179.01
              Detached                  1.00  1200000.00
              Duplex                    1.00  1000000.00
              Flat                      1.00   830952.54
              Ground Flat               1.00   725000.00
              Maisonette                1.00   712500.00
         RENT Apartment                 1.00     3118.15
              Block of Apartments       1.00     3033.00
              Detached                  1.00     3142.00
              Flat                      1.06     3386.77
              House Share               1.00     1473.00

In [120]:
(3118*12)/931179.01 

0.040181318090492614

In [84]:
grouped.index 

MultiIndex([( 1.0,  'BUY'),
            ( 1.0, 'RENT'),
            ( 2.0,  'BUY'),
            ( 2.0, 'RENT'),
            ( 3.0,  'BUY'),
            ( 3.0, 'RENT'),
            ( 4.0,  'BUY'),
            ( 4.0, 'RENT'),
            ( 5.0,  'BUY'),
            ( 5.0, 'RENT'),
            ( 6.0,  'BUY'),
            ( 6.0, 'RENT'),
            ( 7.0,  'BUY'),
            ( 7.0, 'RENT'),
            ( 8.0,  'BUY'),
            ( 8.0, 'RENT'),
            ( 9.0,  'BUY'),
            (10.0,  'BUY'),
            (13.0,  'BUY')],
           names=['BEDROOMS', 'MODE'])

In [85]:
grouped.iloc[grouped.index.get_level_values('MODE') == 'BUY'].round()

avg                median                     count
              BATHROOMS       PRICE BATHROOMS       PRICE PROPERTY TYPE
BEDROOMS MODE                                                          
1.0      BUY        1.0    863059.0       1.0    750000.0           210
2.0      BUY        2.0   1583403.0       2.0   1300000.0           382
3.0      BUY        2.0   2972008.0       2.0   2350000.0           233
4.0      BUY        3.0   4677418.0       3.0   3995000.0            91
5.0      BUY        4.0   9219457.0       4.0   6500000.0            46
6.0      BUY        5.0   8403462.0       6.0   7500000.0            13
7.0      BUY        6.0  16920000.0       6.0  18500000.0             5
8.0      BUY        6.0  11833333.0       6.0  11000000.0             3
9.0      BUY        6.0   8925000.0       6.0   8925000.0             2
10.0     BUY       10.0   6950000.0      10.0   6950000.0             1
13.0     BUY        NaN   9950000.0       NaN   9950000.0             1

In [110]:
rent_ps = grouped.iloc[grouped.index.get_level_values('MODE') == 'RENT'].round(2)['avg']['PRICE']*12 
sales_ps = grouped.iloc[grouped.index.get_level_values('MODE') == 'BUY'].round(2)['avg']['PRICE']
rent_ps.reset_index(drop=True) / sales_ps.iloc[:8].reset_index(drop=True)

0    0.045335
1    0.042139
2    0.046278
3    0.026874
4    0.026642
5    0.069809
6    0.041490
7    0.132710
Name: PRICE, dtype: float64

In [111]:
rent_ps = grouped.iloc[grouped.index.get_level_values('MODE') == 'RENT'].round(2)['median']['PRICE']*12 
sales_ps = grouped.iloc[grouped.index.get_level_values('MODE') == 'BUY'].round(2)['median']['PRICE']
rent_ps.reset_index(drop=True) / sales_ps.iloc[:8].reset_index(drop=True)

0    0.048528
1    0.042000
2    0.045957
3    0.028635
4    0.031961
5    0.027200
6    0.039352
7    0.094309
Name: PRICE, dtype: float64